**Author: Miao Cai**

## Question 1

In [73]:
import os
import numpy as np
import dask.dataframe as dd
import dask.array as da
import dask.delayed
from dask.dot import dot_graph

In [74]:
op = dd.read_csv('healthcare2/OutpatientVisit.csv')
diseasemap = dd.read_csv('healthcare2/DiseaseMap.csv')

# reshape ICD wide to long
oplong = dd.melt(op, id_vars = 'PatientID',
var_name = 'DiagNum', value_name = 'ICD10',
value_vars = ['ICD10_1', 'ICD10_2', 'ICD10_3'])

In [75]:
depression = diseasemap.loc[diseasemap.Condition == 'Depression', 'ICD10'].compute()
depression_str = '|'.join(depression)

In [9]:
# recode multiple depression diagnosis to 1
def f(x):
    if x > 1:
        x = 1
    return x

oplong['depression'] = oplong.ICD10.str.contains(depression_str)
# depression on each patient
q1 = oplong.groupby('PatientID').depression.agg('sum').reset_index()
q1['depression'] = q1.depression.apply(f) # recode

/Users/miaocai/anaconda3/lib/python3.7/site-packages/dask/dataframe/core.py:2259: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  warnings.warn(msg)


### The answer for Q1 a)

In [10]:
q1['depression'].sum().compute()/len(q1['depression'])

0.10718648208469056

### The answer for Q1 b)

In [11]:
def f12(x):
    if x != 0:
        x = 1
    return x

Mortality = dd.read_csv("./healthcare2/Mortality.csv")
p12 = q1.merge(Mortality, how = 'left')
p12['Death'] = p12.DateOfDeath.fillna(0)
p12['Death'] = p12['Death'].apply(f12)
p12.groupby('depression').Death.apply(lambda x: x.sum()/len(x)).reset_index().head()

/Users/miaocai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  # Remove the CWD from sys.path while we load stuff.
/Users/miaocai/anaconda3/lib/python3.7/site-packages/dask/dataframe/core.py:4382: UserWarning: Insufficient elements for `head`. 5 elements requested, only 2 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(msg.format(n, len(r)))


,depression,Death
0,0,0.342834
1,1,0.430199


## Question 2

In [79]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import array, col, explode, \
lit, struct,concat,regexp_extract
from typing import Iterable 
import numpy as np
import pandas as pd

spark = SparkSession.builder\
        .appName("Data Import")\
        .master("local[1]")\
        .getOrCreate()

In [94]:
op = spark.read.csv(
    'healthcare2/OutpatientVisit.csv',
    header=True, inferSchema=True)

Disease = spark.read.csv(
    'healthcare2/DiseaseMap.csv',
    header=True, inferSchema=True)

Mortality = spark.read.csv(
    'healthcare2/Mortality.csv',
    header=True, inferSchema=True)

In [95]:
def melt(
        df: DataFrame, 
        id_vars: Iterable[str], value_vars: Iterable[str], 
        var_name: str="variable", value_name: str="value") -> DataFrame:
    """Convert :class:`DataFrame` from wide to long format."""

    # Create array<struct<variable: str, value: ...>>
    _vars_and_vals = array(*(
        struct(lit(c).alias(var_name), col(c).alias(value_name)) 
        for c in value_vars))

    # Add to the DataFrame and explode
    _tmp = df.withColumn("_vars_and_vals", explode(_vars_and_vals))

    cols = id_vars + [
            col("_vars_and_vals")[x].alias(x) for x in [var_name, value_name]]
    return _tmp.select(*cols)


df2=melt(op, id_vars=['PatientID'], value_vars=['ICD10_1', 'ICD10_2','ICD10_3'])

In [113]:
df2 = df2.withColumnRenamed('value','ICD10')
df3 = df2.join(Disease, on='ICD10',
               how='left_outer')

In [109]:
df4 = df3.filter(df3['Condition'] == 'Depression').select('PatientID').distinct()

Py4JJavaError: An error occurred while calling o1810.showString.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:226)
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.doExecuteBroadcast(BroadcastExchangeExec.scala:146)
	at org.apache.spark.sql.execution.InputAdapter.doExecuteBroadcast(WholeStageCodegenExec.scala:370)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:144)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:140)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:152)
	at org.apache.spark.sql.execution.SparkPlan.executeBroadcast(SparkPlan.scala:140)
	at org.apache.spark.sql.executio

In [ ]:
df4.select('PatientID').distinct().count()
#0.10347109120521172

In [ ]:
df2 = df2.withColumnRenamed('value','ICD10')

df3 = df2.join(Disease,
                   on='ICD10',
                   how='left_outer')

df3.show(5)
df4=df3\
    .filter(df3['Condition'] == 'Depression')
df4.show(5)
nume=df4.select('PatientID').distinct().count()
#0.10347109120521172


#risk of depression among people
nume/deno


In [ ]:
## 2 (b)
df5 = df3.join(Mortality,
                   on='PatientID',
                   how='left_outer')
df5.show(5)
df5 = df5.withColumn("mortality", lit(1))
df5=df5.withColumn("mortality", s_f.when(df5.DateOfDeath.isNull(), lit(0)).otherwise(lit(1)))
df6=df5\
    .filter(df5['Condition'] == 'Depression')
df6.show(5)
DD=df6.filter(df6['mortality'] != 0).select('PatientID').distinct().count()
DN=df6.filter(df6['mortality'] == 0).select('PatientID').distinct().count()

df7=df5\
    .filter(df5['Condition'] != 'Depression')
ND=df7.filter(df7['mortality'] != 0).select('PatientID').distinct().count()
NN=df7.filter(df7['mortality'] == 0).select('PatientID').distinct().count()   

#risk of death among depression
DD/(DD+DN)    
 # 0.10347109120521172  
#risk of death among non-depression
ND/(ND+NN)   
#0.10347109120521172